In [2]:
import os
import os
from loguru import logger
from langchain_deepseek import ChatDeepSeek
import yfinance as yf
from langchain.tools import tool
from dotenv import load_dotenv
from langgraph.graph import END, StateGraph
from dataclasses import dataclass, field
import nest_asyncio
from fastembed import TextEmbedding
from langchain_core.embeddings import Embeddings
_ = load_dotenv()
nest_asyncio.apply()

In [ ]:

embedding = TextEmbedding(model = "sentence-transformers/all-MiniLM-L6-v2", cache_dir = os.path.expanduser("~/.cache/fastembed"))
llm = ChatDeepSeek(model = "deepseek-chat", temperature = 1.3)

In [40]:
@dataclass
class FastEmbed(Embeddings):
    fe: TextEmbedding = field(default_factory=TextEmbedding)

    def embed_query(self, text: str):
        try:
            return list(self.fe.embed(text))[0].tolist()
        except Exception as e:
            print(f"Failed to embed query: {str(e)}")
            raise

    def embed_documents(self, texts: list[str]):
        try:
            return [list(self.fe.embed(texts))[0].tolist()]

        except Exception as e:
            print(f"Failed to embed documents: {str(e)}")
            raise

In [60]:
@tool
def get_stock_price(ticker: str) -> str:
    """Get the current price of a stock based on the given ticker symbol."""
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(period = "1d")
        if data.empty:
            return f"Could not find stock data for {ticker}"
        current_price = data["Close"].iloc[-1]
        return f"Current price of {ticker}: ${current_price:.2f}"
    except Exception as e:
        logger.error(f"Failed to get the stock price for {ticker}: {str(e)}")
        return f"Failed to get the stock price for {ticker}: {str(e)}"


In [61]:
price = get_stock_price("AAPL")
price

'Current price of AAPL: $247.51'

In [ ]:
def get_stock_info(ticker: str) -> str:
    """Get a detailed info about a stock including company name, sector, and market cap."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        sector = info.get("sector", "N/A")
        market_cap = info.get("marketCap", 0)
        pe_ratio = info.get('trailingPE', 'N/A')
        dividend_yield = info.get('dividendYield', 0)

        market_cap_b = market_cap / 1e9 if market_cap else 0
        div_yield_pct = dividend_yield * 100 if dividend_yield else 0

        return (
                f"{company_name} ({ticker})\n"
                f"Sector: {sector}\n"
                f"Market Cap: ${market_cap_b:.2f}B\n"
                f"P/E Ratio: {pe_ratio}\n"
                f"Dividend Yield: {div_yield_pct:.2f}%"
        )
    except Exception as e:
        logger.error(f"Error fetching stock info for {ticker}: {e}")
        return f"Error fetching stock info for {ticker}: {str(e)}"

In [63]:
stock = yf.Ticker("AAPL")
info = stock.info
# info.get("longName", "N/A")
info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '(408) 996-1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and p